# Hackathon 1: descriptive statistics, estimation and bootstrapping

This project illustrates the course LEPL1109 with an industrial applications of statistics. You will analyse the performance of wind farms located in the Walloon Brabant near Beauvechain and in the high Fens, close to Elsenborn. Each wind farm exploits 8 wind turbines. The kinetic power, noted Pk, (in watt) of one wind turbine is calculated with the formula 

Pk (W) = 0.5 x Rho x S x V^3

Where 

S   : is the surface of the circle in square meters with a radius equal to the length of blades (32 m).
Rho : is the air density (“masse volumique”). We consider here an average value of 1.2 kg/m^3
V   : is the speed of wind in m/s.

According to the Betz limit, only 59.3% of the kinetic energy from wind can be used to spin the turbine and generate electricity. In reality, turbines cannot reach the Betz limit, and the efficiency is estimated to 42% of the kinetic power. One wind turbine generates an electric power Pe = 0.42 x Pk. 

The wind turbins are stopped if the wind speed is above 90 km/h.

The file “BeauvechainWind.csv” and “Elsenborn.csv” contains the average daily wind speeds, measured in km/h, at Beauvechain and  Elsenborn (source www. https://www.ecad.eu/)

FILE FORMAT (MISSING VALUE CODE IS -9999):
01-06 SOUID: Source identifier
08-15 DATE : Date YYYYMMDD
17-21 FG   : wind speed in 0.1 m/s
23-27 Q_FG : Quality code for FG (0='valid'; 1='suspect'; 9='missing')


## Report content

•	Grades are granted to the members whose names are in the Jupyter notebook. If your name doesn’t appear on the top of the notebook, you’ll get a 0, even though you are in a group on Moodle.

•	The jupyter notebook must be compiled with printed results and next submitted via moodle. The absence of compiled results (or non-printed values) leads to a lower grade.

## Report submission

•	Deadline, see moodle website. Submission after the deadline will not be accepted.

•	To submit your report, go to the section “APP” on Moodle and the subsection “Soumission du rapport”. You can upload your work there. Once you are sure that it is your final version, click the button “Envoyer le devoir”. It is important that you don’t forget to click on this button ! 

•	Reports that have not been uploaded through Moodle will not be corrected.


## Names and Noma of participants:

Part. 1: Emile VILLETTE (10992000)

Part. 2:

Part. 3:

Part. 4:


In [4]:
import matplotlib.pyplot as plt
import numpy as np
import math
import pandas as pd
from datetime import datetime
from scipy import stats

ModuleNotFoundError: No module named 'seaborn'

In [ ]:
def Pk(V):
    return 0.5 * 1.2 * (math.pi * (32 ** 2)) * ((V/3.6) ** 3) * 8


def Pe(Pk):
    return 0.42 * Pk


def open_csv(filename):
    return pd.read_csv(filename, parse_dates=['DATE'], skipinitialspace=True)


def select_correct_data(data):
    return data[(data.FG > 0) & ((data.Q_FG != 1) & (data.Q_FG != 9))]

In [ ]:
df_be = select_correct_data(open_csv('Beauvechain.csv'))
df_el = select_correct_data(open_csv('Elsenborn.csv'))
df_be = df_be[(datetime(2017, 1, 1) <= df_be['DATE']) & (df_be['DATE'] <= datetime(2021, 1, 1))]
df_el = df_el[(datetime(2017, 1, 1) <= df_el['DATE']) & (df_el['DATE'] <= datetime(2021, 1, 1))]
cdv = np.intersect1d(df_be['DATE'].values, df_el['DATE'].values)
df_be = df_be[df_be['DATE'].isin(cdv)]
df_el = df_el[df_el['DATE'].isin(cdv)]

------------------------------------------------------------------------------------------------------
1.	Write a function computing the electric power capacity (in Mega Watts = 10^6 W), for the two wind farms using wind data from the 1/1/2017 to the 1/1/2021

•	Do not forget to set the production to zero if the wind speed is above 90 km. 

•	Take care to converts units.

•	Remove all days with missing data (error code 1 or 9) for Beauvechain or Elsenborn

------------------------------------------------------------------------------------------------------

In [ ]:
df_be['PowerCapacity'] = Pe(Pk(df_be['FG'])) * (10) ** -6
df_el['PowerCapacity'] = Pe(Pk(df_el['FG'])) * (10) ** -6

In [ ]:
df_be.loc[df_be['FG'] >= 90, 'PowerCapacity'] = 0
df_el.loc[df_el['FG'] >= 90, 'PowerCapacity'] = 0

In [ ]:
df_pcd = df_be.set_index('DATE')[['PowerCapacity']].join(df_el.set_index('DATE')[['PowerCapacity']], lsuffix='_be', rsuffix='_el')
df_pcd.columns = ['Beauvechain power capacity', 'Elsenborn power capacity']
df_pcd

Comment briefly your results



------------------------------------------------------------------------------------------------------------------------

2.	Plot histograms and box plots of electric power capacity (MW) from the 1/1/2017 to the 1/1/2021 (both sites). 

------------------------------------------------------------------------------------------------------------------------

In [ ]:
#histogram of electric power capacities in the given periode
def q2_histogram(df, title=''):
    plt.figure(figsize=(10, 3))
    plt.title(title)
    plt.bar(df['DATE'], df['PowerCapacity'], label='Power capacity')
    plt.plot(df['DATE'], df['PowerCapacity'].rolling(window=30).mean(), color='red', alpha=.7, label='Montly average')
    plt.xlim(df['DATE'].min(), df['DATE'].max())
    plt.ylabel('Power (MW)')
    plt.xlabel('Date')
    plt.legend()

def q2_boxplot(df, title=''):
    plt.figure(figsize=(10, 3))
    plt.title(title)
    plt.boxplot(df['PowerCapacity'],showfliers=False)
    plt.ylabel('Power (MW)')
    plt.xlabel('Date')


q2_histogram(df_be, title='Beauvechain')
q2_histogram(df_el, title='Elsenborn')
q2_boxplot(df_be, title='Beauvechain')
q2_boxplot(df_el, title='Elsenborn')

------------------------------------------------------------------------------------------------------

3.	Compare the following statistics of both sites and draw a first conclusions

•	Average and median powers for each farm (MW)

•	Standard deviations of powers, for each farm (MW)

•	5% and 95% percentiles of powers, for each farm (MW)

The average and standard deviation of the total power capacities (Beauvechain + Elsenborn).

------------------------------------------------------------------------------------------------------

In [ ]:
def q3(df, name=""):
    print(f"{name} farm:")
    print(f" - Average power: {df['PowerCapacity'].mean():.2f} MW")
    print(f" - Median power: {df['PowerCapacity'].median():.2f} MW")
    print(f" - Standard deviation: {df['PowerCapacity'].std():.2f} MW")
    print(f" - Percentile 5: {np.percentile(df['PowerCapacity'], 5):.2f} MW")
    print(f" - Percentile 95: {np.percentile(df['PowerCapacity'], 95):.2f} MW")

q3(df_be, "Beauvechain")
print()
q3(df_el, "Elsenborn")
print()
dfc = df_be.copy()
dfc['PowerCapacity'] += df_el['PowerCapacity'].values
q3(dfc, "Both")

Comment briefly your results




------------------------------------------------------------------------------------------------------
4.	Fit Gamma and Inverse Gaussian distributions to wind speeds (in Km/h) in Beauvechain and Elsenborn. Estimate their parameters by log-likelihood maximization (MLE). Which distribution is the best one? Compare the histograms of winds with the fitted pdf’s on the same plot.
------------------------------------------------------------------------------------------------------

In [ ]:
def q4(df, fct, title=''):
    df_fgc = df.groupby(['FG'])['PowerCapacity'].count()

    X = np.linspace(df['FG'].min(), df['FG'].max(), 1000)
    params = fct.fit(df['FG'].values)
    y = fct.pdf(X, *params)

    plt.figure(figsize=(10, 2))
    plt.title(title)
    plt.xlabel('Wind speed (km/h)')
    plt.ylabel('Rate')
    plt.bar(df_fgc.index, df_fgc.values / df_fgc.sum())
    plt.plot(X, y, color='red')
    
    return params

best_be_params = q4(df_be, stats.gamma, title='Beauvechain (gamma)')
best_el_params = q4(df_el, stats.gamma, title='Elsenborn (gamma)')
q4(df_be, stats.invgauss, title='Beauvechain (inverse gaussian)')
_ = q4(df_el, stats.invgauss, title='Elsenborn (inverse gaussian)')

Comment briefly your results



------------------------------------------------------------------------------------------------------
5.	Compute numerically for both sites, the following statistics 

•	Expected and median powers for each farm (MW)

•	Standard deviation of powers for each farm (MW)

•	5% and 95% percentiles of powers for each farm (MW)

Use the best distributions fitted in Question 4 (not observed values)

------------------------------------------------------------------------------------------------------

In [ ]:
def q5(fct, params, name=""):
    sample_ws = fct.rvs(*params, 1000000)
    sample_pw = Pe(Pk(sample_ws)) * 10**-6
    sample_pw[sample_ws >= 90] = 0
    
    print(f"{name} farm:")
    print(f" - Theorical average power: {sample_pw.mean():.2f} MW")
    print(f" - Theorical median power: {np.median(sample_pw):.2f} MW")
    print(f" - Theorical standard deviation: {np.std(sample_pw):.2f} MW")
    print(f" - Theorical percentile 5: {np.percentile(sample_pw, 5):.2f} MW")
    print(f" - Theorical percentile 95: {np.percentile(sample_pw, 95):.2f} MW")
    
    return sample_pw

_ = q5(stats.gamma, best_be_params, "Beauvechain")
print()
_ = q5(stats.gamma, best_el_params, "Elsenborn")

Comment briefly your results



------------------------------------------------------------------------------------------------------

6.	Same question as Q.4 but this time, you fit the best distribution by the methods of moments MM, (in Python). Compare parameter estimates and plot pdf’s obtained by MLE and MM for both wind farms.

------------------------------------------------------------------------------------------------------

In [ ]:
#code here
def sample_moment(sample, n):
    sm = np.sum([nb**n for nb in sample])
    lgt = len(sample)
    return 1/lgt * sm

def estimate(sample):
    first_moment = sample_moment(sample,1)
    second_moment = sample_moment(sample,2)
    k = first_moment**2/(second_moment - first_moment**2)
    theta = second_moment/first_moment - first_moment
    return k, theta



def q6(df, title=''):
    df_fgc = df.groupby(['FG'])['PowerCapacity'].count()
    X = np.linspace(df["FG"].min(), df["FG"].max(), 1000)

    #method of moments
    k_chapeau , theta_chapeau = estimate(df["FG"].values)
    y_mm = stats.gamma.pdf(x=X, a= k_chapeau, scale=theta_chapeau)

    #MLE method
    params = stats.gamma.fit(df['FG'].values)
    y_mle = stats.gamma.pdf(X, *params)

    plt.figure(figsize=(10, 4))
    plt.title(title)
    plt.xlabel('Wind speed (km/h)')
    plt.ylabel('Rate')
    plt.bar(df_fgc.index, df_fgc.values / df_fgc.sum())
    plt.plot(X, y_mle, color="red", label="MLE estimation")
    plt.plot(X, y_mm, color='green', label="MM estimation")
    plt.legend()

mm_be = q6(df_be, title="Beauvechain method of moments")
mm_el = q6(df_el, title="Elsenborn method of moments")


------------------------------------------------------------------------------------------------------

7.	Bootstrap 1000 times a sample of 500 daily speeds of wind for both wind farms and compute a 5% confidence interval for parameter(s) estimates for the best distribution of Question 4) modelling winds in Beauvechain. How do you interpret the results

------------------------------------------------------------------------------------------------------

In [ ]:
#code here



Comment briefly your results



------------------------------------------------------------------------------------------------------

8.	Let us denote by X the wind speed observed at Beauvechain on a day of the year. The winds observed over 30 days are denoted by (X1,…,X30). They are realizations of X, a random variable with the distribution fitted in Question 4. If M=max{ X1,…,X30},  

•	find the statistical law of M (equation requested, hint use the cdf of X) 

•	plot the pdf and cdf of M. 

•	Determine the wind value, p5, such that  P(M>p5)=95%.


------------------------------------------------------------------------------------------------------

Comment briefly your results



In [ ]:
# Code here
X_i 1 -> 30
Mi = max{x1 - x30}
F_n
f_n
p(M>p) = 95%
soit x appartenant a R (arbitraire)
F_n (x) =P[m<=x] = P[max[x1...x30]<=x] = p( tous les x_i <= x) = produit de tous les x_i = p (X<=x)**30 = F_x(x)**30
F_n = F_x**30
a) f_n = F_n = 30*(F_x**29) * f_x
P t.q. F_n(p) = 5%